In [129]:
%display latex
%display default

## Précision numérique

Par défaut, les nombres flottants et les calculs flottants sont fait avec une précision de 53 bits

In [130]:
a= 3.0
a.parent()

Real Field with 53 bits of precision

In [131]:
a.precision()

53

In [132]:
(1.0/3.0).parent()

Real Field with 53 bits of precision

In [133]:
print(1/3, (1/3).parent())
print( (1/3).n(), (1/3).n().parent())
print(RR(1/3), RR(1/3).parent())


1/3 Rational Field
0.333333333333333 Real Field with 53 bits of precision
0.333333333333333 Real Field with 53 bits of precision


Les nombres flottants sont définis comme instance de l'objet `RealField`. On peut spécifier la précision de cette objet.

In [134]:
R=RealField()
R10=RealField(10)
R200=RealField(200)

R.precision(), R10.precision(), R200.precision()

(53, 10, 200)

In [135]:
a   = R(3.513)
a10 = R10(3.513)
a200= R200(3.513)

a^100, a10^100, a200^100

(3.69670656868857e54,
 3.6e54,
 3.6967065686885807935368708162449489665828918404074416864066e54)

Calcul de $0.3-0.2-0.1$ avec différentes précisions

In [136]:
a,b,c=.3,.2,.1
a10,b10,c10=R10(.3),R10(.2),R10(.1)
a200,b200,c200=R200(.3),R200(.2),R200(.1)
a-b-c,a10-b10-c10,a200-b200-c200

(-2.77555756156289e-17,
 -0.00012,
 7.7787690973264271339300800672251553007378152109014589163764e-62)

**Attention:** 
- si on ne précise pas RealField, la précision est calculée automatiquement.
- si les calculs sont mixtes (opérandes avec différentes précisions), les calculs sont fait dans la plus petite précision

In [137]:
a=0.5555555555554033333333323232323233232111117
a.precision(), a.parent()

(143, Real Field with 143 bits of precision)

In [138]:
a10-b200-c200, a10-b10-c10

(-0.00012, -0.00012)

In [139]:
a= 0.1e-20
b= 0.1
c= 0.3e-10
a+b+c-b,b-b+a+c,a+c+b-b

(3.00000024822111e-11, 3.00000000010000e-11, 3.00000024822111e-11)

**A RETENIR**
> - `RealField(p)` est l'objet définissant les approximations flottantes à précision $p$
> - `numerical_approx(expr, p)` ou `n(expr,p)` ou `expr.n(p)` retourne l'approximation flottante de l'expression `expr` à précision `p`
> - par défaut `RealField(53)` est utilisé (déjà défini dans `RR`)

## Exemple de problème de précision

### calcul de racine carré

In [140]:
a=51
b=a^61
print("l'entier b est codée sur "+str( b.nbits()) + " bits")
r=sqrt(b)
r

l'entier b est codée sur 347 bits


1686961934066707040236155036109474174986501789839001*sqrt(51)

In [141]:
s   = R(sqrt(b))
s200= R200(sqrt(b))
print(floor(s),floor(s200))
floor(s)^2 <= b <(floor(s)+1)^2,\
floor(s200)^2 <= b <=(floor(s200)+1)^2

12047317913813611992216795507023424262776490557440000 12047317913813610588023392961569665056411324497659166


(False, True)

## résolution de système d'équations linéaires 


In [142]:
N=2
MS=MatrixSpace(QQ,N)
VS=VectorSpace(QQ,N)
A=MS.random_element(density=1,num_bound=10,den_bound=10)
b=VS.random_element(num_bound=100,den_bound=100)
V=var(list("x"+ str(i) for i in range(N)))
X=vector(V)
eq=list((A*X)[i]==b[i] for i in range(N))

### La fonction `solve`

elle prend une liste d'équations symboliques et une liste d'inconnues

In [143]:
print("variables:",V)
print("Equations:",eq)
#for i in range(N):
#    print(eq[i])

variables: (x0, x1)
Equations: [-2*x0 + 1/3*x1 == (11/21), -2/3*x0 + 3/2*x1 == (-97/28)]


In [144]:
sol=solve(eq, V)
sol

[[x0 == (-489/700), x1 == (-131/50)]]

Est-il possible de faire la même chose mais de faire le calcul numériquement ?

In [145]:
eqf=list((A.n()*X)[i]==b[i].n() for i in range(N))
print("variables:",V)
print("Equations:",eqf)
#for i in range(N):
#    print(eqf[i])

variables: (x0, x1)
Equations: [-2.00000000000000*x0 + 0.333333333333333*x1 == 0.523809523809524, -0.666666666666667*x0 + 1.50000000000000*x1 == -3.46428571428571]


In [146]:
solf=solve(eqf,V)
solf

[[x0 == (-489/700), x1 == (-131/50)]]

Quoi, le résultat est le même !!!

En fait, `solve` fait appel à une méthode de résolution symbolique. 
Cela implique que les réels sont transformées en nombre rationnel lors de la résolution. 

### l'algèbre linéaire

In [147]:
show(eq)

[-2*x0 + 1/3*x1 == (11/21), -2/3*x0 + 3/2*x1 == (-97/28)]

In [148]:
show(A, (V), b)

[  -2  1/3]
[-2/3  3/2] (x0, x1) (11/21, -97/28)

In [149]:
y=A.solve_right(b)
show(y)

(-489/700, -131/50)

Pareil mais en numérique

In [150]:
Af=A.numerical_approx()
bf=b.n()
show(Af,V,bf)

[ -2.00000000000000  0.333333333333333]
[-0.666666666666667   1.50000000000000] (x0, x1) (0.523809523809524, -3.46428571428571)

In [151]:
yf=Af.solve_right(bf)
show(yf)

(-0.698571428571429, -2.62000000000000)

In [152]:
# difference des solutions
y.n()-yf

(-1.11022302462516e-16, -4.44089209850063e-16)

Tout pareil mais en plus grand

In [153]:
N=80
MS=MatrixSpace(QQ,N)
VS=VectorSpace(QQ,N)
A=MS.random_element(density=1,num_bound=100,den_bound=100)
b=VS.random_element(num_bound=100,den_bound=100)


In [154]:
#print(A,b)
%time y=A.solve_right(b)

CPU times: user 13.2 s, sys: 106 ms, total: 13.3 s
Wall time: 13.4 s


In [155]:
y[0]

-197730178838781139905099057462105891930242788229998362669000658554157431353065606318105983413896123071982986534104383484001333316780311096109346484287114399681498946089645434483868668062745621745932582372750110493141788685354527230708737302905839008202119375914677078998699633355979339217806084905672074873100618985975414122877719821835081428902846861318498598226915934569875445188071335217549087844385612400578903834197660444651281767680154756527804262416850046693165291897929409319626014805413138691853736328988453268555279494093986342507964819180034699164309263928869689099371976431365859643341444030668849824214697138168122031505394190608480522971956640271750583425217475266742566990166602830481437780941789327316035572906131367309543015959423943659143385795007931587587154391484297891153197376757752081539208874037449515910535781582792473873751721509650655315793540229450583961749908409731630332246770045867563533606976487497343757122520130386412286028751881254947737778192750291823382092211772

In [156]:
print("num(y[0])=",y[0].numerator(),'\n')
print("den(y[0])=",y[0].denominator())

num(y[0])= -1977301788387811399050990574621058919302427882299983626690006585541574313530656063181059834138961230719829865341043834840013333167803110961093464842871143996814989460896454344838686680627456217459325823727501104931417886853545272307087373029058390082021193759146770789986996333559793392178060849056720748731006189859754141228777198218350814289028468613184985982269159345698754451880713352175490878443856124005789038341976604446512817676801547565278042624168500466931652918979294093196260148054131386918537363289884532685552794940939863425079648191800346991643092639288696890993719764313658596433414440306688498242146971381681220315053941906084805229719566402717505834252174752667425669901666028304814377809417893273160355729061313673095430159594239436591433857950079315875871543914842978911531973767577520815392088740374495159105357815827924738737517215096506553157935402294505839617499084097316303322467700458675635336069764874973437571225201303864122860287518812549477377781927502918233

In [157]:
Af=A.n()
bf=b.n()
%time yf= Af.solve_right(bf)

CPU times: user 284 ms, sys: 3.6 ms, total: 287 ms
Wall time: 289 ms


In [158]:
print("yf[0]=",yf[0])
#erreur commise par la résolution numérique
y[0]-yf[0], (Af*yf)[0]- bf[0]

yf[0]= -0.344937697019949


(5.31608090881264e-12, -4.24127399867302e-12)

In [159]:
N=20
A=matrix(N,N,lambda i,j : 1/(i+j+1))
b=vector([ZZ.random_element() for i in range(N)])
show(A)
show(b)

20 x 20 dense matrix over Rational Field (use the '.str()' method to see the entries)

(-2, -10, 1, -7, -11, 0, 0, 0, 18, 1, 0, -1, 1, -1, 2, 0, 1, -1, 0, 4)

In [160]:
Af=A.n()
bf=b.n()
%time y = A.solve_right(b)
%time yf= Af.solve_right(bf)

CPU times: user 1.39 ms, sys: 291 µs, total: 1.68 ms
Wall time: 1.43 ms
CPU times: user 5.36 ms, sys: 3 µs, total: 5.37 ms
Wall time: 5.37 ms


In [161]:
### le résultat avec les flottants est complètement faux
print("yf[0]=",yf[0])
(Af*yf)[0], bf[0]

yf[0]= 4.06509636802346e10


(1.74139200000000e6, -2.00000000000000)

In [162]:
### le résultat avec les rationnels est exact
print("y[0]=",y[0])
(A*y)[0],b[0],A*y==b

y[0]= 2243396662525420


(-2, -2, True)

**A RETENIR**
- la fonction `solve` est une méthode de résolution symbolique
- les calculs avec les flottants peuvent être plus rapides mais aussi faux (surtout en algèbre linéaire)
- les calculs avec les rationnels entrainent un grossissement des données mais le résultat est toujours correct

## Limite du calcul symbolique

In [163]:
var('a b c')
e1=a*c+b*c
e2=(a+b)*c
e1.show()
e2.show()
(e2 == e1.collect(c)).show()
(e1 == e2.expand()).show()
bool(e1==e2)

a*c + b*c

(a + b)*c

(a + b)*c == (a + b)*c

a*c + b*c == a*c + b*c

True

In [164]:
e=(a-b)^2-a^2+2*a*b-b^2
e.show()
bool(e==0)

(a - b)^2 - a^2 + 2*a*b - b^2

True

In [165]:
var('x')
e= cos(x)^2+sin(x)^2
e, e.simplify_trig()

(cos(x)^2 + sin(x)^2, 1)

In [166]:
e=sqrt(x^2)
bool(x==e), bool(x==e.simplify_full()),bool(x==e.canonicalize_radical())

(False, False, True)

### Explosion combinatoire des symboles

In [167]:
var('x')
A=Matrix(2,2,[[cos(x),sin(x)],[-sin(x),cos(x)]])
show("A=",A,"  det(A)=",A.determinant())

'A=' [ cos(x)  sin(x)]
[-sin(x)  cos(x)] '  det(A)=' cos(x)^2 + sin(x)^2

In [168]:
N=100
d=(A^N).determinant()
print("d=",d)

#A^N,(A^(N)).expand()

d= 16*(16*(256*(16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (64*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2)^2)*(128*(64*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2)*(16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 + (256*(16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*si

In [169]:
%time print(bool(d==1))
%time print(bool(d.simplify_trig()==1))

True
CPU times: user 4.02 s, sys: 25.1 ms, total: 4.05 s
Wall time: 3.84 s
True
CPU times: user 636 ms, sys: 6.65 ms, total: 643 ms
Wall time: 489 ms


### Pas toujours de solutions

In [170]:
var('x')
(x^2+1).roots()

[(-I, 1), (I, 1)]

In [171]:
E= (x+1)^5 - (x+2)^4
F= (x+1)^5 + (x+2)^4
EF=E*F
print(f"{E=}\n{F=}\n{EF=}")
EF.roots(x)

E=(x + 1)^5 - (x + 2)^4
F=(x + 1)^5 + (x + 2)^4
EF=((x + 1)^5 + (x + 2)^4)*((x + 1)^5 - (x + 2)^4)


[(-1/6*(1/2)^(1/3)*(3*sqrt(23)*sqrt(3) + 97)^(1/3)*(I*sqrt(3) + 1) - 13/3*(1/2)^(2/3)*(-I*sqrt(3) + 1)/(3*sqrt(23)*sqrt(3) + 97)^(1/3) - 1/3,
  1),
 (-1/6*(1/2)^(1/3)*(3*sqrt(23)*sqrt(3) + 97)^(1/3)*(-I*sqrt(3) + 1) - 13/3*(1/2)^(2/3)*(I*sqrt(3) + 1)/(3*sqrt(23)*sqrt(3) + 97)^(1/3) - 1/3,
  1),
 (1/3*(1/2)^(1/3)*(3*sqrt(23)*sqrt(3) + 97)^(1/3) + 26/3*(1/2)^(2/3)/(3*sqrt(23)*sqrt(3) + 97)^(1/3) - 1/3,
  1),
 (-1/2*I*sqrt(3) - 3/2, 1),
 (1/2*I*sqrt(3) - 3/2, 1)]

In [172]:
E= (x+1)^10 - (x+2)^9
F= (x+1)^10 + (x+2)^9
EF=E*F
print(f"{E=}\n{F=}\n{EF=}")
EF.roots(x) ## aucunes solution trouvées alors que c'est un polynôme

E=(x + 1)^10 - (x + 2)^9
F=(x + 1)^10 + (x + 2)^9
EF=((x + 1)^10 + (x + 2)^9)*((x + 1)^10 - (x + 2)^9)


RuntimeError: no explicit roots found

Pour résoudre on passe du calcul symbolique au calcul algébrique.

Pour cela on se place dans la structure mathématique des polynomes qui sont des expressions symboliques ayant de bonne propriétés: en particulier une représentation canonique !!!

Un polynôme est une expression de la forme : $a_0+a_1X+\dots+a_nX^n$ avec $n$ une valeur connue et les coefficients $\{a_0,a_1,\dots,a_n\}$ qui sont tous du même type (Entier, Rationnel, Complexe, ...).

$\Rightarrow$ le vecteur $[a_0,a_1,\dots,a_n]$ est une représentation canonique de ce polynôme.

In [ ]:
# Ensemble des polynomes à coefficient dans QQ
# on choisit la variable X comme symbole des polynômes
PR.<X>=PolynomialRing(QQ) #ou PR.<X>=QQ[]

print("type de x: ",x.parent())
print("type de X: ",X.parent())
bool(X==x)

In [ ]:
print((2*X+4).parent()) 
print((2*x+4).parent()) 
print((2*X+ cos(3)).parent())
print((2*X+0.5).parent()) # attention coercion vers les polynomes à coefficient dans RR
RR

In [ ]:
# on convertit les expressions symboliques vers les polynomes en substituant 
# la variable x par celle des polynômes en X et en construisant le polynôme associé
polyE=PR(E.subs(x=X)) 
polyF=PR(F.subs(x=X))
polyEF=polyE*polyF
show(f"une expression symbolique: {E=}")
show(f"un polynomes: {polyE=}")


In [ ]:
# calcul des racines dans le domaine de définition du polynome (ici Q)
print(f"calcul des racines sur {polyEF.base_ring()}")
polyEF.roots()

In [ ]:
# calcul des racines dans C
R=polyEF.roots(CC)
R

In [ ]:
R[0][0],polyEF(R[0][0])

In [ ]:
polyEF(ComplexField(80)(R[0][0]))

In [ ]:
# calcul de la forme canonique d'un produit de polynômes
E1= (x+1)^100 - (x+2)^99
F1= (x+1)^100 + (x+2)^99
polyE1=PR(E.subs(x=X)) 
polyF1=PR(F.subs(x=X))

# via la representation par arbre (exp symbolique)
%time (E1*F1).expand() 
# via la representation par liste des coefficients
%time polyEF1=polyE1*polyF1 

**A RETENIR**
- le calcul symbolique permet de manipuler n'importe quelles expressions faisant intervenir des symboles et des fonctions 
- pas toujours possible de trouver des solutions avec uniquement des calculs symboliques
- l'utilisation de calcul algèbrique est souvent préférable si on connaît la structure mathématique sous-jacente (Ex. les polynômes, les matrices, ...)
- le calcul algébrique sera la plupart du temps plus rapide
- parfois on retrouve un mélange de calcul algébrique et numérique (résolution dans les complexes si le problème est défini sur les nombres rationnels)

## Les matrices

In [ ]:
A=matrix([[1,2],[3,4]])
B=matrix([[1,2,3],[4,5,6]])
print(A,B)
show(A,B)

In [ ]:
# addition de matrice
show(A+A)
# multiplication par un scalaire
show(3*A , A+A+A)

In [ ]:
show("A=",A)
show("B=",B)

A+B

### matrices et combinaisons linéaires

In [ ]:
# une combinaison linéaire de symboles
var('x y z')
e=[x,y,z]
3*e[0]+2*e[1]+5*e[2]

In [ ]:
u=matrix(1,3,[1,2,3])
v=matrix(3,1,e)
show(u,v)

In [ ]:
# le produit de matrice n*1 par 1*n est une combinaison linéaire
u*v

In [ ]:
# on peut augmenter le nombre de combinaisons linéaires (on ajoute une ligne à gauche)
B=matrix(2,3,[1,2,3,4,5,6])
show(B,v)
show(B*v)

In [ ]:
# on peut augmenter le nombre de combinaisons linéaires (on ajoute une colonne à droite)
var('x,y,z,a,b,c')
V=matrix(3,2,[[x,a],[y,b],[z,c]])
show(V)
show(B*V)

In [ ]:
# ATTENTION: les tailles des matrices doivent correspondre
show(V)
V*V

In [ ]:
# ATTENTION: le produit matriciel n'est pas commutatif
bool(V*B==B*V)

In [ ]:
show(V*B,B*V)

In [ ]:
# On peut transposer une matrice câd intervertir A[i,j] avec A[j,i]
show(V)
show(V.transpose())

In [ ]:
# le produit d'une matrice par sa transposée est toujours bien défini
show(V.transpose()*V)
show(V*V.transpose())

### Les espaces de matrices

In [ ]:
# le premier paramètres est le domaine des coefficients (optionnel)
# si il est présent il impose le type des coefficients
A=matrix([[1,2],[3,4]])
B=matrix(RR,[[1,2/3],[3,4]])
C=matrix([[1,2/3],[3,4]])
A.parent(),B.parent(),C.parent()

In [ ]:
matrix(ZZ,[[1,2/3],[3,4]])

In [ ]:
# le second/troisièmes paramètres sont les dimensions (optionnel)
# si ils sont présents ils imposent le nombre de coefficients (si aucun que des zéros)
A=matrix(ZZ,2,2)
B=matrix(ZZ,2,2,[[4,5],[6,7]])
C=matrix(ZZ,2,2,[4,5,6,7])
show(A)
show(B)
show(C)

In [ ]:
# Comme les matrices vivent dans un espace de matrice on peut définir cet objet
MS23=MatrixSpace(ZZ,2,3)
MS22=MatrixSpace(QQ,2,2)
show(MS22,"\tet\t",MS23)
MS22,MS23

In [ ]:
MS23.dimension() , MS23.dims()

In [ ]:
A=MS22.matrix([1,2,3,4])
A.parent()

In [ ]:
MS22.random_element()
show(_)

In [ ]:
MS10=MatrixSpace(ZZ,10,10)
MS10.random_element(density=1,x=0,y=100)
show(_)

In [ ]:
MS10=MS10.change_ring(QQ)
MS10.random_element(density=0.5,num_bound=100,den_bound=100)
show(_)

### un peu de calcul algébrique matriciel

In [ ]:
N=10
q=17
MS=MatrixSpace(GF(q),N,N)
A=MS.random_element()
show("A=",A)
show(f"det(A)={A.det()}")

In [ ]:
B=A.change_ring(ZZ)
show("B=",B)
show(f"det(B)={B.det()}")

In [ ]:
B.det()%q == A.det()

In [ ]:
show(A^-1)

In [ ]:
show(B^-1)

In [ ]:
(B^(-1)%q), A^-1
show(_)

In [ ]:
# Expliquer le résultat obtenu
(B^(-1)%q) ==  A^-1

In [ ]:
(B^(-1)%q).parent(),(A^-1).parent()

In [ ]:
(B^(-1)%q) ==  (A^-1).change_ring(B.base_ring())

In [ ]:
(B^(-1)).change_ring(A.base_ring()) ==  (A^-1)